<a href="https://colab.research.google.com/github/prakharsrivastava/powerbi/blob/main/Pyspark__important_funtions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
from google.colab import drive

drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/chapter2/


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/chapter2


In [70]:
!pip install findspark 
!pip install  pyspark



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [71]:
import findspark
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
spark = SparkSession.builder.master("local[1]").appName("architectinaction.com").getOrCreate()
sc= spark.sparkContext

In [ ]:
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9,10])

In [ ]:
rddCollect = rdd.collect()
print("Number of Partitions: "+str(rdd.getNumPartitions()))
print("Action: First element: "+str(rdd.first()))
print(rddCollect)

In [ ]:

emptyRDD = sc.emptyRDD()
emptyRDD2 = rdd=sc.parallelize([])

print("is Empty RDD : "+str(emptyRDD2.isEmpty()))

In [ ]:

# Create spark session with local[5]
rdd = sc.parallelize(range(0,20))
print("From local[5] : "+str(rdd.getNumPartitions()))

# Use parallelize with 6 partitions
rdd1 = sc.parallelize(range(0,25), 6)
print("parallelize : "+str(rdd1.getNumPartitions()))

rddFromFile = sc.textFile("readme.html",10)
print("TextFile : "+str(rddFromFile.getNumPartitions()))

In [ ]:

rdd1.saveAsTextFile("partition")


In [ ]:
# Using repartition
rdd2 = rdd1.repartition(4)
print("Repartition size : "+str(rdd2.getNumPartitions()))
rdd2.saveAsTextFile("re-partition")

In [ ]:
# Using coalesce()
rdd3 = rdd1.coalesce(4)
print("Repartition size : "+str(rdd3.getNumPartitions()))
rdd3.saveAsTextFile("coalesce")

In [ ]:

from pyspark.sql.types import StructType,StructField, StringType, IntegerType,ArrayType,MapType
from pyspark.sql.functions import col,struct,when

data = [("James","","Smith","36636","M",3000),
    ("Michael","Rose","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Maria","Anne","Jones","39192","F",4000),
    ("Jen","Mary","Brown","","F",-1)
  ]

spark.createDataFrame(data).show(truncate=False)
schema = StructType([ 
    StructField("firstname",StringType(),True), 
    StructField("middlename",StringType(),True), 
    StructField("lastname",StringType(),True), 
    StructField("id", StringType(), True), 
    StructField("gender", StringType(), True), 
    StructField("salary", IntegerType(), True) 
  ])
 
df = spark.createDataFrame(data=data,schema=schema)
df.printSchema()
df.show(truncate=False)


In [ ]:

structureData = [
    (("James","","Smith"),"36636","M",3100),
    (("Michael","Rose",""),"40288","M",4300),
    (("Robert","","Williams"),"42114","M",1400),
    (("Maria","Anne","Jones"),"39192","F",5500),
    (("Jen","Mary","Brown"),"","F",-1)
  ]

spark.createDataFrame(structureData).show(truncate=False)
structureSchema = StructType([
        StructField('name', StructType([
             StructField('firstname', StringType(), True),
             StructField('middlename', StringType(), True),
             StructField('lastname', StringType(), True)
             ])),
         StructField('id', StringType(), True),
         StructField('gender', StringType(), True),
         StructField('salary', IntegerType(), True)
         ])

df2 = spark.createDataFrame(data=structureData,schema=structureSchema)
df2.printSchema()
df2.show(truncate=False)


updatedDF = df2.withColumn("OtherInfo", 
    struct(col("id").alias("identifier"),
    col("gender").alias("gender"),
    col("salary").alias("salary"),
    when(col("salary").cast(IntegerType()) < 2000,"Low")
      .when(col("salary").cast(IntegerType()) < 4000,"Medium")
      .otherwise("High").alias("Salary_Grade")
  ))
#.drop("id","gender","salary")

updatedDF.printSchema()
updatedDF.show(truncate=False)



In [ ]:
#Broadcast variables are read-only shared variables that are cached and available on all nodes in a cluster 

In [ ]:

states = {"NY":"New York", "CA":"California", "FL":"Florida"}
broadcastStates = sc.broadcast(states)

data = [("James","Smith","USA","CA"),
    ("Michael","Rose","USA","NY"),
    ("Robert","Williams","USA","CA"),
    ("Maria","Jones","USA","FL")
  ]

rdd = sc.parallelize(data)
df=spark.createDataFrame(rdd)
def state_convert(code):
    return broadcastStates.value[code]

result = rdd.map(lambda x: (x[0],x[1],x[2],state_convert(x[3]))).collect()
print(result)

In [ ]:
df.show()

In [ ]:
#filteDf= rdd.where((rdd['state'].isin(broadcastStates.value)))


In [ ]:

from pyspark.sql.functions import lit

df=df.withColumn("name",lit("architectInAction"))
df.show()

In [ ]:
from pandas.core.reshape.api import concat
df=df.withColumn("name2",lit("name"))
df.show()



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat,col
data = [('James','','Smith','1991-04-01','M',3000),
  ('Michael','Rose','','2000-05-19','M',4000),
  ('Robert','','Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]

df = spark.createDataFrame(data)
df2=df.withColumn("name3",concat(col("_1"),lit("777")))
             
df2.show(truncate=False)

In [ ]:
df.select("name").show()
df.select(df.name).show()
df.select(df["name"]).show()

#By using col() function
from pyspark.sql.functions import col
df.select(col("name")).show()

#Select columns by regular expression
df.select(df.colRegex("`^.*name*`")).show()

In [ ]:
df.distinct().show()
df.select("_6").distinct().show()
df.dropDuplicates().show()

In [ ]:
df.distinct().show()
print(df.select("_6").distinct().count())
df.dropDuplicates().show()

In [ ]:
df=df.select(countDistinct("_2", "_6"))
df.show()

In [ ]:
#group by
#agg() – Using groupBy() agg() function, we can calculate more than one aggregate at a time.

In [ ]:
df.groupBy("_5").sum("_6").show(truncate=False)


In [ ]:
df.groupBy("_5").min("_6").show()

In [ ]:
df.groupBy("_5") .agg(sum("_6").alias("sum_salary"),avg("_6").alias("avg_salary") ).show(truncate=False)

In [ ]:
df.groupBy("_5") .agg(sum("_6").alias("sum_salary"),avg("_6").alias("avg_salary") ).where(col("avg_salary") >= 3000).show(truncate=False)

In [ ]:
df.join(df,df._5 ==  df._5,"inner") \
     .show(truncate=False)

In [ ]:
df = df.union(df)
df.show()

In [ ]:
columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders")]

df = spark.createDataFrame(data=data,schema=columns)
def upperCase(str):
    return str.upper()

upperCaseUDF = udf(lambda z:upperCase(z),StringType())    

df.withColumn("Cureated Name", upperCaseUDF(col("Name"))) \
.show(truncate=False)



In [ ]:
import pyspark.pandas as ps
import numpy as np

# Create a DataFrame
psdf = ps.DataFrame(df)
psdf


In [ ]:

def add(data):
   return data[0] + "ffff"
   
psdf = psdf.apply(add,axis=1)
psdf.head()

In [73]:
simpleData = (("Java",4000,5), \
    ("Python", 4600,10),  \
    ("Scala", 4100,15),   \
    ("Scala", 4500,15),   \
    ("PHP", 3000,20),  \
  )
columns= ["CourseName", "fee", "discount"]

# Create DataFrame
df = spark.createDataFrame(data = simpleData, schema = columns)
df.printSchema()
df.show(truncate=False)

# Custom transformation 1
from pyspark.sql.functions import upper
def to_upper_str_columns(df):
    return df.withColumn("CourseName",upper(df.CourseName))

# Custom transformation 2
def reduce_price(df,reduceBy):
    return df.withColumn("new_fee",df.fee - reduceBy)

# Custom transformation 3
def apply_discount(df):
    return df.withColumn("discounted_fee",  \
             df.new_fee - (df.new_fee * df.discount) / 100)

# transform() usage
df2 = df.transform(to_upper_str_columns) \
        .transform(reduce_price,1000) \
        .transform(apply_discount) 
                
df2.show()


root
 |-- CourseName: string (nullable = true)
 |-- fee: long (nullable = true)
 |-- discount: long (nullable = true)

+----------+----+--------+
|CourseName|fee |discount|
+----------+----+--------+
|Java      |4000|5       |
|Python    |4600|10      |
|Scala     |4100|15      |
|Scala     |4500|15      |
|PHP       |3000|20      |
+----------+----+--------+

+----------+----+--------+-------+--------------+
|CourseName| fee|discount|new_fee|discounted_fee|
+----------+----+--------+-------+--------------+
|      JAVA|4000|       5|   3000|        2850.0|
|    PYTHON|4600|      10|   3600|        3240.0|
|     SCALA|4100|      15|   3100|        2635.0|
|     SCALA|4500|      15|   3500|        2975.0|
|       PHP|3000|      20|   2000|        1600.0|
+----------+----+--------+-------+--------------+



In [ ]:

data = ["Project Gutenberg’s",
        "Alice’s Adventures in Wonderland",
        "Project Gutenberg’s",
        "Adventures in Wonderland",
        "Project Gutenberg’s"]
rdd=spark.sparkContext.parallelize(data)
rdd.collect()


In [ ]:
rdd2=rdd.flatMap(lambda x: x.split(" "))
rdd2.collect()
 

In [ ]:
df2.show()

In [80]:
#overwrite
df2.write.mode('append').option("header",True) \
        .option("maxRecordsPerFile", 5) \
        .partitionBy("fee") \
        .mode("append") \
        .csv("course")

DataFrame[CourseName: string, fee: bigint, discount: bigint, new_fee: bigint, discounted_fee: double]

In [ ]:
data = [("James","M",60000),("Michael","M",70000),
        ("Robert",None,400000),("Maria","F",500000),
        ("Jen","",None)]
columns = ["name","gender","salary"]
df = spark.createDataFrame(data = data, schema = columns)
df.show()

In [ ]:
df = df.withColumn("new_gender", when(df.gender == "M","Male")
                                 .when(df.gender == "F","Female")
                                 .when(df.gender.isNull() ,"")
                                 .otherwise(df.gender))
df.show()

In [ ]:
from pyspark.sql.functions import expr, col

#Using Case When on withColumn()
df3 = df.withColumn("new_gender", expr("CASE WHEN gender = 'M' THEN 'Male' " + 
               "WHEN gender = 'F' THEN 'Female' WHEN gender IS NULL THEN ''" +
               "ELSE gender END"))
df3.show(truncate=False)

In [ ]:

df.createOrReplaceTempView("EMP")
spark.sql("select name, CASE WHEN gender = 'M' THEN 'Male' " + 
               "WHEN gender = 'F' THEN 'Female' WHEN gender IS NULL THEN ''" +
              "ELSE gender END as new_gender from EMP").show()

In [98]:
data =[("James ","","Smith","36636","M",3000),
              ("Michael ","Rose","","40288","M",4000),
              ("Robert ","","Williams","42114","M",4000),
              ("Maria ","Anne","Jones","39192","F",4000),
              ("Jen","Mary","Brown","","F",-1)]
columns=["firstname","middlename","lastname","dob","gender","salary"]
df=spark.createDataFrame(data,columns)
df.write.mode("overwrite").parquet("people.parquet")
parDF1=spark.read.parquet("people.parquet")
parDF1.createOrReplaceTempView("parquetTable")
parDF1.printSchema()
parDF1.show(truncate=False)


root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|dob  |gender|salary|
+---------+----------+--------+-----+------+------+
|James    |          |Smith   |36636|M     |3000  |
|Michael  |Rose      |        |40288|M     |4000  |
|Robert   |          |Williams|42114|M     |4000  |
|Maria    |Anne      |Jones   |39192|F     |4000  |
|Jen      |Mary      |Brown   |     |F     |-1    |
+---------+----------+--------+-----+------+------+



In [99]:

parkSQL = spark.sql("select * from ParquetTable where salary >= 4000 ")
parkSQL.show(truncate=False)


df.write.partitionBy("gender","salary").mode("overwrite").parquet("people.parquet")

parDF2=spark.read.parquet("people.parquet/gender=M")
parDF2.show(truncate=False)


+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|dob  |gender|salary|
+---------+----------+--------+-----+------+------+
|Michael  |Rose      |        |40288|M     |4000  |
|Robert   |          |Williams|42114|M     |4000  |
|Maria    |Anne      |Jones   |39192|F     |4000  |
+---------+----------+--------+-----+------+------+

+---------+----------+--------+-----+------+
|firstname|middlename|lastname|dob  |salary|
+---------+----------+--------+-----+------+
|James    |          |Smith   |36636|3000  |
|Michael  |Rose      |        |40288|4000  |
|Robert   |          |Williams|42114|4000  |
+---------+----------+--------+-----+------+



In [100]:
df = spark.read.json("flights_sample.json")
df.printSchema()
df.show()

root
 |-- ARR_DELAY: long (nullable = true)
 |-- ARR_TIME: string (nullable = true)
 |-- CANCELLED: boolean (nullable = true)
 |-- CRS_ARR_TIME: string (nullable = true)
 |-- CRS_DEP_TIME: string (nullable = true)
 |-- DEP_DELAY: long (nullable = true)
 |-- DEP_TIME: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEST_AIRPORT_SEQ_ID: string (nullable = true)
 |-- DISTANCE: string (nullable = true)
 |-- DIVERTED: boolean (nullable = true)
 |-- FL_DATE: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- ORIGIN_AIRPORT_SEQ_ID: string (nullable = true)
 |-- TAXI_IN: long (nullable = true)
 |-- TAXI_OUT: long (nullable = true)
 |-- UNIQUE_CARRIER: string (nullable = true)
 |-- WHEELS_OFF: string (nullable = true)
 |-- WHEELS_ON: string (nullable = true)

+---------+--------+---------+------------+------------+---------+--------+----+-------------------+--------+--------+----------+------+---------------------+-------+--------+--------------+----------+---

In [101]:

columns = ["id", "name","age","gender"]

# Create DataFrame 
data = [(1, "James",30,"M"), (2, "Ann",40,"F"),
    (3, "Jeff",41,"M"),(4, "Jennifer",20,"F")]
sampleDF = spark.sparkContext.parallelize(data).toDF(columns)

# Create Hive Internal table
sampleDF.write.mode('overwrite') \
         .saveAsTable("employee")

# Read Hive table
df = spark.read.table("employee")
df.show()

+---+--------+---+------+
| id|    name|age|gender|
+---+--------+---+------+
|  1|   James| 30|     M|
|  2|     Ann| 40|     F|
|  3|    Jeff| 41|     M|
|  4|Jennifer| 20|     F|
+---+--------+---+------+

